In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.en import English
import numpy as np
import pandas as pd

In [2]:
nlp = English()
nlp.add_pipe('sentencizer')

In [6]:
from MainScript import dataHuggingFace

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 96
Current browser version is 98.0.4758.81 with binary path C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	Ordinal0 [0x00626903+2517251]
	Ordinal0 [0x005BF8E1+2095329]
	Ordinal0 [0x004C2848+1058888]
	Ordinal0 [0x004E0B9D+1182621]
	Ordinal0 [0x004DCA60+1165920]
	Ordinal0 [0x004DA2CF+1155791]
	Ordinal0 [0x0050AAAF+1354415]
	Ordinal0 [0x0050A71A+1353498]
	Ordinal0 [0x0050639B+1336219]
	Ordinal0 [0x004E27A7+1189799]
	Ordinal0 [0x004E3609+1193481]
	GetHandleVerifier [0x007B5904+1577972]
	GetHandleVerifier [0x00860B97+2279047]
	GetHandleVerifier [0x006B6D09+534521]
	GetHandleVerifier [0x006B5DB9+530601]
	Ordinal0 [0x005C4FF9+2117625]
	Ordinal0 [0x005C98A8+2136232]
	Ordinal0 [0x005C99E2+2136546]
	Ordinal0 [0x005D3541+2176321]
	BaseThreadInitThunk [0x769DFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77197A9E+286]
	RtlGetAppContainerNamedObjec

In [ ]:
import pandas as pd
df = pd.DataFrame()
for all_news in dataHuggingFace:
    df1 = pd.DataFrame(all_news)
    df = pd.concat([df, df1])
    df = df.reset_index(drop = True)
    nan_value = float("NaN")
    df.replace("", nan_value, inplace=True)
    df.dropna(subset = ["content"], inplace=True)
df

In [ ]:
import re
for i in range(len(df['content'])):
    text = df['content'][i]
    df['content'][i] = re.sub(r'\s+', ' ', text)
df

In [ ]:
generalSummary = []
for text_corpus in df['content']:
    doc = nlp(text_corpus.replace("\n", ""))
    sentences = [sent.text.strip() for sent in doc.sents]
    # Let's create an organizer which will store the sentence ordering to later reorganize the 
    # scored sentences in their correct order
    sentence_organizer = {k:v for v,k in enumerate(sentences)}
    # Let's now create a tf-idf (Term frequnecy Inverse Document Frequency) model
    tf_idf_vectorizer = TfidfVectorizer(min_df=2,  max_features=None, 
                                        strip_accents='unicode', 
                                        analyzer='word',
                                        token_pattern=r'\w{1,}',
                                        ngram_range=(1, 3), 
                                        use_idf=1,smooth_idf=1,
                                        sublinear_tf=1,
                                        stop_words = 'english')
    # Passing our sentences treating each as one document to TF-IDF vectorizer
    tf_idf_vectorizer.fit(sentences)
    # Transforming our sentences to TF-IDF vectors
    sentence_vectors = tf_idf_vectorizer.transform(sentences)
    # Getting sentence scores for each sentences
    sentence_scores = np.array(sentence_vectors.sum(axis=1)).ravel()

    # Sanity checkup
    print(len(sentences) == len(sentence_scores))
    # Getting top-n sentences
    N = 3
    summary = " "
    if len(sentences) < 3:
        summary = " ".join(sentences)
    else:
        top_n_sentences = [sentences[ind] for ind in np.argsort(sentence_scores, axis=0)[::-1][:N]]
        # Let's now do the sentence ordering using our prebaked sentence_organizer
        # Let's map the scored sentences with their indexes
        mapped_top_n_sentences = [(sentence,sentence_organizer[sentence]) for sentence in top_n_sentences]
        print("Our top_n_sentence with their index: \n")
        for element in mapped_top_n_sentences:
            print(element)

        # Ordering our top-n sentences in their original ordering
        mapped_top_n_sentences = sorted(mapped_top_n_sentences, key = lambda x: x[1])
        ordered_scored_sentences = [element[0] for element in mapped_top_n_sentences]

        # Our final summary
        summary = " ".join(ordered_scored_sentences)
    generalSummary.append(summary)

In [ ]:
df['generalSummary'] = generalSummary
del df['content']
df


In [ ]:
from datetime import date
today = date.today()
# dd/mm/YY
d1 = today.strftime("%d-%m-%Y")
output = 'AIS_data(Scraping+Summrization)_'+d1+'.csv'


In [ ]:
df.to_csv(output, encoding='utf-8-sig', index=False)